In [87]:
import pandas as pd
import numpy as np
import time
import os

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam

In [67]:
data = pd.read_csv('input_df_2020-08-09-1936.csv', index_col=0)
data.drop(['date','day_num'],axis=1,inplace=True)
for i in range(len(data)):
    if data['volt sdev 5'][i]==0:
        data.drop(i, inplace=True)

In [68]:
fails = data[data['fail']==1]
no_fails = data[data['fail']==0]

rn=1
no_fails1 = sklearn.utils.shuffle(no_fails, random_state=rn).reset_index(drop=True)[0:691]

In [77]:
df1 = no_fails1.append(fails).reset_index(drop=True)

In [82]:

x = df1.drop(['fail'], axis =1)
y = df1['fail']

In [83]:
x

machineID      volt    rotate  pressure  vibration  volt sdev 5  \
0          95.0  0.788659  0.917665  0.646675   0.678084     0.016724   
1          49.0  0.779832  0.911478  0.646189   0.637008     0.016221   
2          59.0  0.771652  0.898868  0.683414   0.653500     0.007466   
3          88.0  0.773620  0.925425  0.645974   0.641989     0.014443   
4          47.0  0.786385  0.909629  0.643498   0.629805     0.009246   
...         ...       ...       ...       ...        ...          ...   
1377       99.0  0.782218  0.899631  0.649852   0.647956     0.015392   
1378       99.0  0.791816  0.862671  0.713363   0.647604     0.014788   
1379      100.0  0.774922  0.893963  0.655674   0.648252     0.047373   
1380      100.0  0.769071  0.911992  0.660314   0.675930     0.059436   
1381      100.0  0.764024  0.941853  0.642380   0.682325     0.010040   

      rotate sdev 5  pressure sdev 5  vibration sdev 5  days_last_maint  \
0          0.017262         0.008176          0.026757                1   
1          0.026714         0.008056          0.013102               14   
2          0.008517         0.006281          0.014493               12   
3          0.028935         0.016470          0.018524                6   
4          0.063915         0.013324          0.081634                4   
...             ...              ...               ...              ...   
1377       0.014248         0.079728          0.017007               15   
1378       0.081331         0.051971          0.093047               15   
1379       0.015139         0.010986          0.012486               15   
1380       0.027457         0.012343          0.031383               15   
1381       0.103284         0.015051          0.025563               30   

      days_last_fail  model  
0                 15      2  
1                 13      1  
2                 56      3  
3                 50      4  
4                 18      2  
...              ...    ...  
1377              29      1  
1378              14      1  
1379              42      4  
1380             209      4  
1381              89      4  

[1382 rows x 12 columns]

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 0, stratify = y)
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [136]:
#build ANN

model = Sequential()
model.add(Conv1D(32, 1, activation='relu', input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64, 1, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(1, activation='sigmoid'))

from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(lr = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [137]:
epochs = 10
history = model.fit(x_train, y_train, epochs = epochs, validation_data = (x_test, y_test), verbose = 1)

Train on 1174 samples, validate on 208 samples
Epoch 1/10
1174/1174 [==============================] - 2s 1ms/sample - loss: 0.7850 - accuracy: 0.5843 - val_loss: 0.5812 - val_accuracy: 0.8029
Epoch 2/10
1174/1174 [==============================] - 0s 200us/sample - loss: 0.6438 - accuracy: 0.6576 - val_loss: 0.6437 - val_accuracy: 0.6154
Epoch 3/10
1174/1174 [==============================] - 0s 172us/sample - loss: 0.5753 - accuracy: 0.7155 - val_loss: 0.5913 - val_accuracy: 0.7308
Epoch 4/10
1174/1174 [==============================] - 0s 164us/sample - loss: 0.5122 - accuracy: 0.7624 - val_loss: 0.5300 - val_accuracy: 0.8269
Epoch 5/10
1174/1174 [==============================] - 0s 196us/sample - loss: 0.4937 - accuracy: 0.7658 - val_loss: 0.4912 - val_accuracy: 0.8462
Epoch 6/10
1174/1174 [==============================] - 0s 178us/sample - loss: 0.4807 - accuracy: 0.7760 - val_loss: 0.4532 - val_accuracy: 0.8221
Epoch 7/10
1174/1174 [==============================] - 0s 181us/sa

(12, 1)

In [139]:
no_fails1_dev = sklearn.utils.shuffle(no_fails, random_state=rn).reset_index(drop=True)[692:720].reset_index(drop=True)
x1_dev = no_fails1_dev.drop(['fail'], axis=1)

In [150]:
#verify on devset
i=10
input1 = np.array(x1_dev[i:i+1])
input11 = input1.reshape(12,1)
model.predict(input11.reshape(1,12,1))

array([[0.68880457]], dtype=float32)

In [163]:
#verifying on the fails
#this is extremely flawed as it is inputting data that was in the test or train set. 
#need to remove some data from fails train / test to make dev set

i=5
x2_dev = fails.reset_index(drop=True).drop(['fail'],axis=1)[i:i+1]
input2 = np.array(x2_dev)
input22 = input2.reshape(12,1)
model.predict(input22.reshape(1,12,1))

array([[0.69112927]], dtype=float32)